Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos um valor práximo de zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



<h4 style='text-align:justify'> GERAL </h4>
<p style="text-align: justify"> O problema abordado é a respeito de aplicação de restrição na busca, isto é, temos uma mochila com um número $n$ de itens disponíveis, cada um com um peso e um valor associado. A restrição do exercício consiste no fato de que a mochilha tem a capacidade de carregar um número $p$ de quilogramas e se ultrapassarmos esse peso, temos como resultado a mochilha rasgada e a perda de todos os nossos itens, logo, temos como objetivo encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade máxima delimitada. </p>
<p style="text-align: justify"> O enunciado nos permite observar que temos um problema de maximização, porque queremos os itens mais valiosos que não ultrapassem o valor estabelecido pela mochila. Essa determinação de que a mochila possui peso e valor, nos permite utilizar de outra postulação importante: os dicionários. Para quem não se lembra, os dicionários são coleções de itens que tem seus elementos armazenados de forma não ordenada, de modo geral, seus elementos contém uma chave e valor. </p>
<p style="text-align: justify"> Essas características nos permitem desenvolver uma estratégia similar a abordada nos experimentos iniciais - caixas binárias; isto é, se um objeto estiver presente na mochila, o indivíduo terá um valor representando isso e se não, terá outro valor. A própria escrita de se e não se, nos permite utilizar de outra estratégia para a resolução do código, o uso de condicionais if e else para a determinação e análise do problema. Acoplando todas as ideias citadas, podemos construir um algoritmmo de tal forma que selecionado os objetos do dicionários, constroi-se uma lista (porque esta pode ser ordenada) que represente a mochila, contendo todos os objetos sendo identificada a presença ou não do objeto na mochilha pelos valores de 0 (não tem o objeti) e 1 (temos o objeto). </p>
<p style="text-align: justify"> Dada essa estratégia, podemos fazer combinações que analisem qual o melhor fitness dos individuos que solucionem o nosso problema, novamente utilizando de conceitos dos Algoritmos Genéticos como Fitness, Cruzamento e Mutação. Vamos ver um pouco mais sobre essa lógica? </p>


<h4 style='text-align:justify'> PASSO 1 </h4>
<p style="text-align: justify"> De modo a realizar o algoritmo genético, determina-se as constantes gerais do problema como o tamanho da população, número de gerações, a probabilidade de cruzamento e mutação, o limite de peso e o dicionário de possíveis objetos com seus pesos e valores. </p>

In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

<h4 style='text-align:justify'> PASSO 2 </h4>
<p style="text-align: justify"> Em seguida, desenvolve-se as as funções objetivo da população - recebe o indivíduo e retorna o seu valor de aptidão. </p>

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

<h4 style='text-align:justify'> PASSO 3 </h4>
<p style="text-align: justify"> Estabelecidas as constantes e funções, a primeira linha de código é responsável pela criação da população a partir das constantes estabelecidas, isto é, uma população com tamanho máximo de 50 individuos e com o número de objetos que é representado pelo dicionário que atribui suas características de peso e valor. Após a criação da população, estabelece-se o fitness como infinito e como o nosso problema é caracterizado como um problema de maximização, queremos encontrar o fitness com maior número. Em seguida, o nosso código irá percorrer em loop pelo número total de gerações estabelecido inicialmente, de modo aos indivíduos que formaram a população troquem as informações dos seus genes segundo o cruzamento e mutação estabelecido. </p>
<p style="text-align: justify">  Esses cruzamentos e mutações irão ser avaliadas pelo fitness que irá retornar o valor de aptidão da combinação, nos permitindo identificar a melhor resposta que solucione o problema determinado, isto é, colocar na mochilha sem exceder o peso, a melhor combinação de itens de peso e valor. </p>

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 13900 | Peso: 10.1
Maior valor: 15500 | Peso: 13.5
Maior valor: 17900 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Harry Potter: ele voltou, confia!
+ Quadrinho super raro do Aranha-Homem da vida real
+ energético
+ tablet
+ teclado musical
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 17900 dinheiros e peso de 14.1 unidades de massa.


**Desafio**: resolva o experimento considerando uma busca em grade para encontrar a melhor resposta.



Fica para a próxima, Dani!

## Conclusão




<p style="text-align: justify"> Neste experimento, temos como situação-problema, resolver o problema da mochila que consiste em encontrar um conjunto de itens que maximize o valor contido dentro da mochila de forma que respeite o limite de peso da mochila. Ressalta-se que no problema da mochila há um número de  $n$ de itens disponíveis, cada um com um peso e um valor associado. Contextualizado o problema, observa-se a característica dele, isto é, temos um problema de maximização com restrição. </p>
<p style="text-align: justify"> De modo a cumprir a restrinção, temos que conduzir o nosso algoritimo genético a fimm de se desenvolver indivudos válidos que solucionem o nosso problema, nossa estratégia consiste em desenvolver uma função objetivo que utilize de operadores para desenvolver genes/individuos válidos que realizem o nosso objetivo: a maximização dentro da nossa limitação. A função objetivo será utilizada no processo de seleção, sendo utilizado o fitness para nos sabermos uma melhor opção de genes. </p>
<p style="text-align: justify"> Um dos problemas iniciais a esse problema consiste em como associar os valores do dicionário de modo ordenado, a fim de resolve-lo, utiliza-se uma estrátegia similar ao abordado em experimentos anteriores, como o experimento de caixa binária, identificando as possibilidades dos itens dentro da mochila por meio de uma operação de 0 (não tem o item) e 1(tem o item). Feita essa operação, temos uma função responsável por somar os valores dos itens e o peso de cada objeto, a fim de controlar a limitação, utiliza-se de condicionais como if e else para que a somatória cumpra o requisito de peso máximo, caso ela não cumpra, atribui-se o valor ao código de 0.01, que nos afirma que a mochila rasgou e a operação não é maximizada da forma desejada. </p>



## Playground



Muito obrigada pela aula, Dani :)